# Testing  model with XAI methods 

In [24]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

In [25]:
import calendar
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, f1_score, precision_score, recall_score, roc_auc_score, accuracy_score
from sklearn.pipeline import FunctionTransformer, Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder, PowerTransformer
from xgboost import XGBClassifier

## Defining model

In [26]:
TARGET_VALUE = 'y'

def get_training_data():
    df = pd.read_csv("datasource/bank_data_model.csv")
    return df.iloc[:,1:]

def get_validation_data():
    df =  pd.read_csv("datasource/bank_data_validate.csv")
    return df.iloc[:,1:]

def get_categorical_features(dataFrame): 
    return dataFrame.drop(columns=get_numerical_features(dataFrame)).columns.values

def get_numerical_features(dataFrame):
    return dataFrame.select_dtypes(include=np.number).columns.values

def get_numerical_features_transformer():
    class DayMonthTransformer(BaseEstimator,TransformerMixin):
        def fit(self, X, y=None):
            return self
        
        def transform(self, X):
            X_transformed = X.copy()
            month_num_dict = {month.lower(): index for index, month in enumerate(calendar.month_abbr) if month}
            X_transformed['month'] = X_transformed['month'].map(month_num_dict)
            X_transformed['year'] = np.repeat(2024, repeats= X_transformed.shape[0])
            X_transformed['day_of_the_year'] = pd.to_datetime(X_transformed[["year","month","day"]], format='%d/%m/%y', errors='coerce')
            X_transformed['day_of_the_year'] = X_transformed['day_of_the_year'].apply(lambda x: x.dayofyear)
            return X_transformed.drop(columns=["day", "month", "year"])
        
        def get_feature_names_out(self, input_features = None):
            return ['day_of_the_year']
        
    return ColumnTransformer(
        transformers=[
            ("duration", PowerTransformer(method="yeo-johnson"), ["duration"]),
            ("age", Pipeline( steps = [
                                ("age_power_transform", PowerTransformer(method="box-cox")),
                                ("age_function_transform", FunctionTransformer(lambda col : col + abs(np.min(col)), feature_names_out='one-to-one'))
                                ]
                            ),
            ["age"]),
            ("balance", Pipeline( steps = [
                                ("balance_function_transform_min", FunctionTransformer(lambda col : col + abs(np.min(col)),feature_names_out='one-to-one')),
                                ("balance_function_transform_log", FunctionTransformer(np.log1p, feature_names_out='one-to-one')),
                                ]
                            ),
            ["balance"]),
            ("day_of_the_year", DayMonthTransformer(), ["day","month"])
            ]
        )
                        
def get_categorical_features_transformer():
    return ColumnTransformer(
        transformers= [
            ("yes_no_variables",  OrdinalEncoder(), ["default", "housing", "loan"]),
            ("sorted_variables", Pipeline(
                steps=[
                    ("ordinal", OrdinalEncoder()),
                    ("min_max_scaler", MinMaxScaler())
                ]
            ),
            ["education", "marital", "poutcome"]),
            ("unsorted_variables", OneHotEncoder(), ["job", "contact"]),
        ]
    )  

def get_features_transformer():
    return ColumnTransformer(
        transformers = [
            ("num", get_numerical_features_transformer(), ["duration","balance", "age", "day", "month"]),
            ("cat", get_categorical_features_transformer(), ["default", "housing", "loan", "education", "marital", "poutcome", "job", "contact"]),
            ("others", FunctionTransformer(feature_names_out='one-to-one'), ["campaign", "pdays","previous"])
        ]
    )

def get_estimator(weights):
    return XGBClassifier(booster= 'gbtree', 
                         eta= 0.10987358313376537,
                         max_delta_step= 1.446953199095078, 
                         max_depth= 4, 
                         min_child_weight= 1.666061864206202, 
                         objective= 'binary:logistic', 
                         scale_pos_weight= weights, 
                         tree_method= 'exact')
def get_model(weights):
    return Pipeline(
        steps = [
            ("preprocessor", get_features_transformer()),
            ("estimator", get_estimator(weights))
        ]
    )

def printMetrics(test, pred): 
    cm = confusion_matrix(test,pred)
    cm_display = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = [False, True])
    cm_display.plot()
    print(f"accuracy : {accuracy_score(test, pred)}")
    print(f"f1       : {f1_score(test, pred)}")
    print(f"precision: {precision_score(test, pred)}")
    print(f"recall   : {recall_score(test, pred)}")
    print(f"roc_auc  : {roc_auc_score(test,pred)}")

In [27]:
from sklearn.calibration import LabelEncoder
from sklearn.model_selection import train_test_split

df = get_training_data()
le = LabelEncoder()
df[TARGET_VALUE] = le.fit_transform(df[TARGET_VALUE])

X_train, X_test = train_test_split(
    df,
    test_size=0.2,
    stratify=df[TARGET_VALUE],
    random_state=42,
)

y_train, y_test = X_train[TARGET_VALUE], X_test[TARGET_VALUE]
X_train, X_test = X_train.drop(columns=TARGET_VALUE), X_test.drop(columns=TARGET_VALUE)

In [28]:
positive = y_train.sum()
negative = y_train.count() - positive
clf = get_model(negative/positive)

In [29]:
clf.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  ColumnTransformer(transformers=[('duration',
                                                                                   PowerTransformer(),
                                                                                   ['duration']),
                                                                                  ('age',
                                                                                   Pipeline(steps=[('age_power_transform',
                                                                                                    PowerTransformer(method='box-cox')),
                                                                                                   ('age_function_transform',
                                                                                                    FunctionTransformer(feature_names_out='one-to-one',
                                                                                                                        func=<function get_numerical_features_transf...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None,
                               max_delta_step=1.446953199095078, max_depth=4,
                               max_leaves=None,
                               min_child_weight=1.666061864206202, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, ...))])

## Dalex

In [30]:
import dalex as dx


In [31]:
exp = dx.Explainer(clf, X_test, y_test)

Preparation of a new explainer is initiated

  -> data              : 724 rows 16 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 724 values
  -> model_class       : xgboost.sklearn.XGBClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7ea9db372020> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.00051, mean = 0.214, max = 0.976
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.951, mean = -0.0993, max = 0.999
  -> model_info        : package sklearn

A new explainer has been created!


In [32]:
mp = exp.model_performance(model_type = 'classification')
mp.result

,recall,precision,f1,accuracy,auc
XGBClassifier,0.686747,0.459677,0.550725,0.871547,0.904066


### ROC 

In [33]:
mp.plot(geom="roc")

In [34]:
mp.plot()

### Feature importance

In [35]:
vi_grouped = exp.model_parts(variable_groups = {
    'age': ['age'], 
    'job': ['job'], 
    'marital': ['marital'], 
    'education': ['education'], 
    'default': ['default'], 
    'balance': ['balance'], 
    'housing': ['housing'], 
    'loan': ['loan'], 
    'contact': ['contact'], 
    'date': ['day', 'month'], 
    'duration': ['duration'], 
    'campaign': ['campaign'], 
    'pdays': ['pdays'], 
    'previous': ['previous'], 
    'poutcome': ['poutcome']
}
)
vi_grouped.result

,variable,dropout_loss,label
0,age,0.092314,XGBClassifier
1,marital,0.095542,XGBClassifier
2,_full_model_,0.095934,XGBClassifier
3,job,0.096201,XGBClassifier
4,campaign,0.096760,XGBClassifier
5,previous,0.096857,XGBClassifier
6,default,0.096985,XGBClassifier
7,loan,0.098090,XGBClassifier
8,housing,0.098243,XGBClassifier
9,education,0.098594,XGBClassifier


In [36]:
vi_grouped.plot()

### Aggregated Profiles

In [14]:
num_var = ['age', 'balance', 'duration', 'campaign', 'pdays',
       'previous']
pdp_num = exp.model_profile(type = 'partial', variables=num_var, label = "pdp", N = 500)

ale_num = exp.model_profile(type = 'accumulated', variables=num_var, label="ale", N = 500)

Calculating ceteris paribus:   0%|          | 0/6 [00:00<?, ?it/s]/home/michal/Dokumenty/python_dodatkowy/lib/python3.11/site-packages/dalex/predict_explanations/_ceteris_paribus/utils.py:104: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[20.  20.6 21.2 ... 78.8 79.4 80. ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

Calculating ceteris paribus:  17%|█▋        | 1/6 [00:00<00:01,  4.87it/s]/home/michal/Dokumenty/python_dodatkowy/lib/python3.11/site-packages/dalex/predict_explanations/_ceteris_paribus/utils.py:104: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-1164.    -731.91  -299.82 ... 41180.82 41612.91 42045.  ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

Calculating ceteris paribus:  33%|███▎      | 2/6 [00:00<00:00,  6.59it/s]/home/michal/Dokumen

In [15]:
pdp_num.plot(ale_num)

In [16]:
cat_var = ['job', 'marital', 'education', 'default', 'housing',
       'loan', 'contact', 'poutcome']

pdp_cat = exp.model_profile(type = 'partial', variable_type='categorical',
                            variables = cat_var, label="pdp")

ale_cat = exp.model_profile(type = 'accumulated', variable_type='categorical',
                            variables = cat_var, label="ale")

Calculating accumulated dependency: 100%|██████████| 8/8 [00:00<00:00, 40.65it/s]


In [17]:
ale_cat.plot(pdp_cat)

### Testing predictions for specific cases

In [45]:
blue_collar_worker = pd.DataFrame(
    {   
        'age': [38],
        'job': ['blue-collar'],
        'maritial': ['married'],
        'education': ['tertiary'],
        'default': ['no'],
        'balance': [20000],
        'housing': ['yes'],
        'loan': ['yes'],
        'contact': ['cellular'],
        'day': [16],
        'month': ['may'],
        'duration': [185],
        'campaign': [1],
        'pdays': [330],
        'previous': [1],
        'poutcome': ['success']
    },
    index=["blue_collar_worker"],
)

student = pd.DataFrame(
    {   
        'age': [20],
        'job': ['student'],
        'maritial': ['single'],
        'education': ['tertiary'],
        'default': ['no'],
        'balance': [350],
        'housing': ['no'],
        'loan': ['no'],
        'contact': ['cellular'],
        'day': [16],
        'month': ['may'],
        'duration': [90],
        'campaign': [-1],
        'pdays': [0],
        'previous': [0],
        'poutcome': ['unknown']
    },
    index=["student"],
)


grandpa = pd.DataFrame(
    {   
        'age': [65],
        'job': ['retired'],
        'maritial': ['married'],
        'education': ['secondary'],
        'default': ['no'],
        'balance': [40000],
        'housing': ['no'],
        'loan': ['no'],
        'contact': ['cellular'],
        'day': [16],
        'month': ['may'],
        'duration': [380],
        'campaign': [-1],
        'pdays': [20],
        'previous': [0],
        'poutcome': ['unknown']
    },
    index=["grandpa"],
)

young_businessman = pd.DataFrame(
    {   
        'age': [20],
        'job': ['entrepreneur'],
        'maritial': ['single'],
        'education': ['secondary'],
        'default': ['no'],
        'balance': [70000],
        'housing': ['no'],
        'loan': ['no'],
        'contact': ['cellular'],
        'day': [1],
        'month': ['may'],
        'duration': [200],
        'campaign': [-1],
        'pdays': [20],
        'previous': [0],
        'poutcome': ['unknown']
    },
    index=["young_businessman"],
)


In [46]:
sh_blue_collar_worker = exp.predict_parts(blue_collar_worker, type="shap", B=50, label=blue_collar_worker.index[0])
sh_student = exp.predict_parts(student, type="shap", B=50, label=student.index[0])
sh_grandpa = exp.predict_parts(grandpa, type="shap", B=50, label=grandpa.index[0])
young_businessman = exp.predict_parts(young_businessman, type="shap", B=50, label=young_businessman.index[0])



In [47]:
sh_john.plot([sh_student, sh_grandpa, young_rich_guy])
